In [1]:
import pandas as pd

In [2]:
tms_acutal_dir = "../../data/actual/TMS_Actual.csv"
tms_pred_dir = "../../data/pred/TMS_Pred.csv"
flow_acutal_dir = "../../data/actual/FLOW_Actual.csv"
flow_pred_dir = "../../data/pred/FLOW_pred.csv"
weather_dir = "../../data/actual/Weather.csv"

In [3]:
tms_actual = pd.read_csv(tms_acutal_dir)
tms_actual.head()

,SYS_TIME,TOC_VU,PH_VU,SS_VU,FLUX_VU,TN_VU,TP_VU
0,2024-08-26 15:09,4.1,7.2,0.9,3912.0,11.923,0.039
1,2024-08-26 15:10,4.1,7.2,0.8,3917.0,11.923,0.039
2,2024-08-26 15:11,4.1,7.1,0.9,3923.0,11.923,0.039
3,2024-08-26 15:12,4.1,7.1,0.9,3927.0,11.923,0.039
4,2024-08-26 15:13,4.1,7.1,2.2,3932.0,11.923,0.039


In [4]:
tms_pred = pd.read_csv(tms_pred_dir)
tms_pred.head()

,SYS_TIME,TOC_VU,PH_VU,SS_VU,FLUX_VU,TN_VU,TP_VU
0,2024-12-13 19:12,0.9,6.8,1.3,4655.0,0.656,0.061
1,2024-12-13 19:13,1.0,6.8,1.3,4658.0,0.656,0.061
2,2024-12-13 19:14,1.0,6.8,1.3,4661.0,0.656,0.061
3,2024-12-13 19:15,1.0,6.8,1.3,4664.0,0.656,0.061
4,2024-12-13 19:16,1.0,6.8,1.3,4669.0,0.656,0.061


In [5]:
flow_acutal = pd.read_csv(flow_acutal_dir)
flow_acutal.head()

,data_save_dt,flow_TankA,flow_TankB,level_TankA,level_TankB,SYS_TIME
0,202509022353,230.4000,230.2500,3.55225,3.54900,2025-09-02 23:53
1,202509022354,229.9875,230.1000,3.54700,3.54575,2025-09-02 23:54
2,202509022355,229.5000,230.0625,3.54825,3.54475,2025-09-02 23:55
3,202509022356,229.8375,230.4750,3.54125,3.54250,2025-09-02 23:56
4,202509022357,229.9125,230.0250,3.54050,3.54050,2025-09-02 23:57


In [6]:
flow_pred = pd.read_csv(flow_pred_dir)
flow_pred.head()

,SYS_TIME,Q_in
0,2024-12-13 18:14,396.360
1,2024-12-13 18:15,403.108
2,2024-12-13 18:16,419.579
3,2024-12-13 18:17,432.523
4,2024-12-13 18:18,450.369


In [7]:
weather = pd.read_csv(weather_dir)
weather.head()

,PREDICT_DATE,1시간 강수량 범주(1mm),기온
0,2024-09-11 15:00,0.5,27
1,2024-09-11 16:00,1,28
2,2024-09-11 17:00,1,28
3,2024-09-11 18:00,0,27
4,2024-09-11 19:00,0,27


In [13]:
# 방법 1: 기대하는 전체 범위와 비교
def check_1min_completeness(df):
    # 전체 1분 단위 범위 생성
    full_range = pd.date_range(start=df.index.min(), 
                                end=df.index.max(), 
                                freq='1min')
    
    # 누락된 시간 찾기
    missing = full_range.difference(df.index)
    
    print(f"전체 기간: {df.index.min()} ~ {df.index.max()}")
    print(f"기대 행 수: {len(full_range)}")
    print(f"실제 행 수: {len(df)}")
    print(f"누락 행 수: {len(missing)}")
    
    if len(missing) == 0:
        print("✓ 모든 1분 단위 데이터 존재")
    else:
        print(f"✗ {len(missing)}개 누락")
        print(f"누락된 시간 (처음 10개):\n{missing[:10]}")
    
    return missing


In [14]:
# 사용
flow = pd.read_csv('../../data/actual/FLOW_Actual.csv')
flow['SYS_TIME'] = pd.to_datetime(flow['SYS_TIME'])
flow = flow.set_index('SYS_TIME')
missing_times = check_1min_completeness(flow)

전체 기간: 2025-09-02 23:53:00 ~ 2025-12-03 10:39:00
기대 행 수: 131687
실제 행 수: 130732
누락 행 수: 955
✗ 955개 누락
누락된 시간 (처음 10개):
DatetimeIndex(['2025-09-03 01:20:00', '2025-09-03 01:57:00',
               '2025-09-03 03:13:00', '2025-09-03 17:30:00',
               '2025-09-03 18:21:00', '2025-09-03 22:58:00',
               '2025-09-04 03:13:00', '2025-09-04 06:56:00',
               '2025-09-04 08:11:00', '2025-09-04 09:26:00'],
              dtype='datetime64[us]', freq=None)


In [12]:
# 인덱스 차이 확인
time_diff = flow.index.to_series().diff()
non_1min = time_diff[time_diff != pd.Timedelta('1min')]

if len(non_1min) == 0:
    print("✓ 모든 간격이 1분")
else:
    print(f"✗ 1분이 아닌 간격 {len(non_1min)}개 발견")
    print(non_1min.head())


✗ 1분이 아닌 간격 934개 발견
SYS_TIME
2025-09-02 23:53:00               NaT
2025-09-03 01:21:00   0 days 00:02:00
2025-09-03 01:58:00   0 days 00:02:00
2025-09-03 03:14:00   0 days 00:02:00
2025-09-03 17:31:00   0 days 00:02:00
Name: SYS_TIME, dtype: timedelta64[us]


In [17]:
# 1분 단위로 reindex (누락된 시간은 NaN으로 채움)
full_range = pd.date_range(start=flow.index.min(), 
                            end=flow.index.max(), 
                            freq='1min')
flow_complete = flow.reindex(full_range)

# 누락 확인
print(f"NaN 개수: {flow_complete.isna().sum()}")


NaN 개수: data_save_dt    955
flow_TankA      955
flow_TankB      955
level_TankA     955
level_TankB     955
dtype: int64


In [18]:
# 인덱스 간격 통계
time_diff = flow.index.to_series().diff()
print(time_diff.value_counts())
# 1분이 대부분이면 정상


SYS_TIME
0 days 00:01:00    129798
0 days 00:02:00       927
0 days 00:03:00         2
0 days 00:07:00         2
0 days 00:06:00         1
0 days 00:08:00         1
Name: count, dtype: int64


In [15]:
# 사용
tms = pd.read_csv('../../data/actual/TMS_Actual.csv')
tms['SYS_TIME'] = pd.to_datetime(tms['SYS_TIME'])
tms = tms.set_index('SYS_TIME')
missing_times = check_1min_completeness(tms)

전체 기간: 2024-08-26 15:09:00 ~ 2025-09-29 05:23:00
기대 행 수: 573975
실제 행 수: 514551
누락 행 수: 59424
✗ 59424개 누락
누락된 시간 (처음 10개):
DatetimeIndex(['2024-08-27 10:16:00', '2024-08-29 17:06:00',
               '2024-08-29 17:30:00', '2024-08-29 21:11:00',
               '2024-08-29 21:12:00', '2024-08-29 21:13:00',
               '2024-08-29 21:14:00', '2024-08-29 21:15:00',
               '2024-08-29 21:16:00', '2024-08-29 21:17:00'],
              dtype='datetime64[us]', freq=None)


In [16]:
# 인덱스 차이 확인
time_diff = tms.index.to_series().diff()
non_1min = time_diff[time_diff != pd.Timedelta('1min')]

if len(non_1min) == 0:
    print("✓ 모든 간격이 1분")
else:
    print(f"✗ 1분이 아닌 간격 {len(non_1min)}개 발견")
    print(non_1min.head())


✗ 1분이 아닌 간격 9096개 발견
SYS_TIME
2024-08-26 15:09:00               NaT
2024-08-27 10:17:00   0 days 00:02:00
2024-08-29 17:07:00   0 days 00:02:00
2024-08-29 17:31:00   0 days 00:02:00
2024-08-30 09:15:00   0 days 12:05:00
Name: SYS_TIME, dtype: timedelta64[us]


In [19]:
# 1분 단위로 reindex (누락된 시간은 NaN으로 채움)
full_range = pd.date_range(start=tms.index.min(), 
                            end=tms.index.max(), 
                            freq='1min')
tms_complete = tms.reindex(full_range)

# 누락 확인
print(f"NaN 개수: {tms_complete.isna().sum()}")


NaN 개수: TOC_VU     147455
PH_VU      147455
SS_VU      147455
FLUX_VU    147450
TN_VU      147451
TP_VU      147455
dtype: int64


In [21]:
# 인덱스 간격 통계
time_diff = tms.index.to_series().diff()
print(time_diff.value_counts())
# 1분이 대부분이면 정상


SYS_TIME
0 days 00:01:00     505455
0 days 00:02:00       8854
0 days 00:03:00        152
0 days 00:04:00         23
0 days 00:06:00         10
0 days 00:05:00          8
0 days 00:08:00          5
0 days 00:07:00          5
0 days 00:10:00          4
0 days 00:12:00          3
0 days 00:09:00          3
0 days 00:22:00          2
0 days 00:14:00          2
0 days 00:18:00          2
0 days 12:05:00          1
5 days 00:48:00          1
0 days 00:13:00          1
0 days 00:29:00          1
0 days 11:50:00          1
12 days 19:17:00         1
0 days 00:31:00          1
1 days 22:33:00          1
3 days 01:12:00          1
0 days 00:24:00          1
0 days 03:21:00          1
0 days 15:35:00          1
0 days 16:05:00          1
0 days 23:07:00          1
0 days 16:35:00          1
0 days 01:39:00          1
1 days 20:49:00          1
0 days 00:25:00          1
3 days 20:54:00          1
1 days 17:37:00          1
0 days 01:11:00          1
0 days 00:37:00          1
Name: count, dtype: